In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import re
import time

In [9]:
options = EdgeOptions()
options.use_chromium = True
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.set_capability("acceptInsecureCerts", True)
#Try to connect if the edgeriver version is 108
options.binary_location =r'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'

In [3]:
dicParfum={}
listeMarque=[]
listeIngredient=[]

In [4]:
def Sephora(options):
    driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
    driver.get("https://www.sephora.fr/")
    cookies=driver.find_element(By.XPATH,'//*[@id="footer_tc_privacy_button_2"]')
    cookies.click()
    for i in range (0,10):
        div=0
        driver.get("https://www.sephora.fr/parfum-meilleures-ventes/?srule=Sorting%20Rule%20-%20Best%20Sellers&start=0&sz=24")
        id=str(i+1)
        #print("top : "+ id )
        try:
            pathparfum="/html/body/div[2]/div[3]/div[2]/div[3]/div[2]/div[2]/ul/li["+id+"]"
            #//*[@id="search-result-items"]/li[1]/div
            Parfum=driver.find_element(By.XPATH, pathparfum)
        except :
            pathparfum="/html/body/div[1]/div[3]/div[2]/div[3]/div[2]/div[2]/ul/li["+id+"]"
            Parfum=driver.find_element(By.XPATH, pathparfum)
        if Parfum.get_attribute('class')=='grid-tile':
            marque=driver.find_element(By.XPATH,pathparfum+"/div/div[3]/div/a/span").get_attribute('innerText')
            #print(marque)
            listeMarque.append(marque)
            Parfum.click()
            #nom du parfum      
            nomParfum=driver.find_element(By.CLASS_NAME,'product-name.product-name-bold').get_attribute("innerText")
            #print(nomParfum)
            dicParfum[i]=nomParfum
            #liste des ingredients
            ingredient=driver.find_element(By.CLASS_NAME,'ingredients-content.tabs-panel').get_attribute('innerText')
            #print('Ingredient : '+ ingredient)
            listeIngredient.append(ingredient)
            #print("\n")
    driver.close()
        

<ipython-input-4-47e148e7b33a>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)


# Gestion des Ingredients

In [6]:
 def IngrefientForme():
    phrase="Cette liste d'ingrédients peut faire l'objet de modifications, veuillez consulter l'emballage du produit acheté."
    Ingredient=[]
    for i in listeIngredient:
        ouisticram=i.replace("\n",'')
        ouisticram=ouisticram.replace(phrase,'')
        ousticram=ouisticram.replace(phrase,'')
        if "INGREDIENTS" in ouisticram:
            champenfeu=ouisticram.split(':')
            champenfeu=champenfeu[-1]
        else:
            champenfeu=ouisticram
        if "●" in champenfeu :
            simiabraz=champenfeu.split("●")
        elif " | " in champenfeu:
            simiabraz=champenfeu.split(" | ")
        elif ' ,' in champenfeu:
            simiabraz=champenfeu.split(" ,")
        elif ',' in champenfeu:
            simiabraz=champenfeu.split(",")
        elif " • " in champenfeu:
            simiabraz=champenfeu.split(" • ")
        else:
            print(simiabraz)
        Ingredient.append(simiabraz)


    Ingredients=[]
    for i in Ingredient:
        for j in i :
            if j.startswith('#'):
                ij=j.split(' ')
                j=ij[1]
            if j.startswith(' '):
                j=j[1:]
            if j.endswith(' '):
                j=j[:-1]
            Ingredients.append(j)


Recherche des ingrédients et de leur potentiel mise en garde

In [8]:
setIngredient=set(Ingredients)

In [9]:
descIng={}
Allergene=[]
endo=[]

In [10]:
def incibeauty(options):
    driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
    driver.get("https://incibeauty.com/ingredients")
    cookies=driver.find_element(By.XPATH,"/html/body/div[6]/div[1]/div/div[2]/button")
    cookies.click()
    for i in setIngredient:
        driver.get("https://incibeauty.com/ingredients")
        search=driver.find_element(By.ID,'searchInci')
        search.click()
        search.clear()
        search.send_keys(i)
        count=0
        url=False
        b=True
        while b:
            count+=1
            search.send_keys(Keys.ENTER)
            if driver.current_url!="https://incibeauty.com/ingredients":
                b=False
                url=True
            if count>=10:
                b=False
        if url :
            listeInfo= WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/section[1]/div/div/div/ul')))
            if 'Allergène' in listeInfo.get_attribute('innerText'):
                Allergene.append(i)
            if 'Perturbateur endocrinien' in listeInfo.get_attribute('innerText'):
                endo.append(i)
            try:
                aSavoir=driver.find_element(By.XPATH,'/html/body/section[1]/div/div/div/div[2]/div')
                descIng[i]=aSavoir.get_attribute('innerText')
            except:
                descIng[i]="Pas d'information supplémentaire"

    driver.close()

<timed exec>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3


Wall time: 2min 34s


# Gestion des engagements de marques

In [14]:
marqueEco={listeMarque[i]: 'Sans engagement' for i in range(len(listeMarque))}
marqueEco

{'GIVENCHY': 'Sans engagement',
 'DIOR': 'Sans engagement',
 'CHANEL': 'Sans engagement',
 'YVES SAINT LAURENT': 'Sans engagement',
 'PACO RABANNE': 'Sans engagement',
 'PRADA': 'Sans engagement'}

In [15]:
def engagement(options):
    driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
    driver.get("https://sciencebasedtargets.org/companies-taking-action#dashboard")
    time.sleep(5)
    cookies=driver.find_element(By.CLASS_NAME,"font-bold.text-center.rounded-full.appearance-none.o-btn.border-3.border-currentColor.flex-shrink-0.is-current")
    cookies.click()
    perpage=Select(driver.find_element(By.XPATH,'//*[@id="per-page"]'))
    #recherche la marque
    inputs=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[1]/div/div[2]/label/input")
    for i in marqueEco.keys():
        inputs.send_keys(i)
        time.sleep(5)
        try :
            #click sur view more pour avoir accès à la target
            driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div/div[2]/label/span/span").click()
            target=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div/div[2]/div/div[3]/div[2]").get_attribute("innerText")
            marqueEco[i]=target
        except:
            marqueEco[i]="Sans engagement"
        inputs.clear()
    driver.close()

<timed exec>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3


beurk
beurk
beurk
Wall time: 42 s
